In [2]:
import sys
sys.path.append('..')

from src.dataset import load_and_preprocess_dataset

IMAGE_PIXEL_SIZE = 64
DEVICE = "mps"
BATCH_SIZE = 32 

train_dl = load_and_preprocess_dataset(
    path="tkarr/sprite_caption_dataset",
    split="train",
    image_pixel_size=IMAGE_PIXEL_SIZE,
    batch_size=BATCH_SIZE,
)

valid_dl = load_and_preprocess_dataset(
    path="tkarr/sprite_caption_dataset",
    split="valid",
    image_pixel_size=IMAGE_PIXEL_SIZE,
    batch_size=BATCH_SIZE,
)

In [8]:

import hashlib

train_checksums = {}
valid_checksums = {}

for batch in train_dl:
    for tensor in batch['tensor']:
        cksum = hashlib.md5(tensor.numpy()).hexdigest()
        train_checksums[cksum] = 1

valid_checksums_found = 0
for batch in valid_dl:
    for tensor in batch['tensor']:
        cksum = hashlib.md5(tensor.numpy()).hexdigest()
        if cksum in train_checksums:
            valid_checksums_found += 1

print(f"Found {valid_checksums_found} valid checksums in train set out of {len(valid_checksums)}")

Found 711 valid checksums in train set out of 0
